In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys #to be able to click/input
from selenium.webdriver.support.ui import WebDriverWait #wait excplicit
from selenium.webdriver.support import expected_conditions as EC #wait explicit
from selenium.webdriver.common.by import By #find_element
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from tabulate import tabulate
import pandas as pd
from pandas.errors import InvalidIndexError
import numpy as np

In [2]:
driver = webdriver.Chrome()

url = 'https://www.tiket.com/hotel/search?room=1&adult=1&id=yogyakarta-province-108001534490276304&type=REGION&q=Provinsi%20Yogyakarta&checkin=2024-09-13&checkout=2024-09-14'

driver.get(url)

In [60]:
# Preparation
tiket_hotel = []
prog = 0
driver.execute_script("window.scrollTo(0, 0);")

card = driver.find_element(By.CSS_SELECTOR, 'div[data-test-id="virtuoso-item-list"]')

# Scraping Loops
for i in range(0,1000):
    while True:
        try:
            hotel = card.find_element(By.CSS_SELECTOR, f'div[data-index="{i}"]')
            break
        except NoSuchElementException as scroll_needed:
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(1.5)
        
    # Name
    name = hotel.find_element(By.CSS_SELECTOR, 'h3.ProductDetails_product_name__yOjso.Text_text__3MOfx.Text_size_b2__zYdnA.Text_weight_bold__zuwFa').text
    # Star
    try:
        star_list = hotel.find_element(By.CSS_SELECTOR, 'div.ProductStarRating_container__kbGyc')
        star = star_list.find_elements(By.TAG_NAME, 'svg')
    except NoSuchElementException as villa_star:
        try:
            star_list = hotel.find_element(By.CSS_SELECTOR, 'div[data-testid="nha-star-rating-container"]')
            star = star_list.find_elements(By.CSS_SELECTOR, 'img[src="https://s-light.tiket.photos/t/01E25EBZS3W0FY9GTG6C42E1SE/t_htl-mobile/core-fe/2023/05/08/69fbc27c-cf72-4e85-bc82-35a91e88085c-1683512112124-e14dc83cd5704efbb41d7eba80517bcc.png"]')
        except NoSuchElementException as no_star:
            star = []
            
    # Price
    price = hotel.find_element(By.CSS_SELECTOR, 'div.Text_text__3MOfx.Text_variant_price__Cb9JA.Text_size_b1__4egl5.Text_align_right__xCYwN.Text_weight_bold__zuwFa').text
    
    # Append
    hotel_list = {'Nama Hotel': name,
                  'Harga per malam': price,
                  'Bintang': len(star)
    }
    tiket_hotel.append(hotel_list)
    
    prog += 1
    if prog % 50 == 0:
        print(f"There is {prog} hotel information retrieved")

There is 50 hotel information retrieved
There is 100 hotel information retrieved
There is 150 hotel information retrieved
There is 200 hotel information retrieved
There is 250 hotel information retrieved
There is 300 hotel information retrieved


KeyboardInterrupt: 

In [61]:
df = pd.DataFrame(tiket_hotel)
df

,Nama Hotel,Harga per malam,Bintang
0,Casa Coco Resort,IDR 330.341,3
1,Radika Paradise Villa & Cottage,IDR 737.602,3
2,Cordia Hotel Yogyakarta - Hotel Dalam Bandara,IDR 456.396,3
3,Sahid Raya Hotel & Convention Yogyakarta,IDR 473.451,4
4,The Rich Jogja Hotel,IDR 714.049,4
...,...,...,...
295,Rumah Singgah Peziarah Ganjuran,IDR 94.837,1
296,Griya Erlangga Homestay,IDR 77.913,1
297,Villa Watu Kodok,IDR 1.227.271,3
298,"De Laxston Hotel Yogyakarta, Azana Hotels Coll...",IDR 600.261,3


In [62]:
df.to_csv("Tiket.com Yogyakarta Hotel.csv", index=False)